# Exercise 10: Prompt Template Variations

**Goal:** The retrieval pipeline is identical across all runs — only the prompt template changes.
This isolates the effect of *instructions* on generation quality.

| # | Variant | Core instruction |
|---|---|---|
| 1 | **Minimal** | No instructions — just context + question |
| 2 | **Strict grounding** | Answer ONLY from context; say so if absent |
| 3 | **Citation** | Quote exact passages that support the answer |
| 4 | **Permissive** | Use context but also your own knowledge |
| 5 | **Structured output** | Facts first, then synthesise |

**For each variant × query this notebook records:**
- The raw answer
- Four evaluation scores (accuracy, groundedness, helpfulness, citation quality) via GPT-4o Mini judge
- Saves everything to CSV and produces comparison charts

**Documentation questions:**
- Which prompt produces the most *accurate* answers?
- Which produces the most *useful* answers?
- Is there a trade-off between strict grounding and helpfulness?

---
## Setup

In [ ]:
try:
    ip = get_ipython()
    ip.run_line_magic('pip', 'install -q torch transformers sentence-transformers faiss-cpu pymupdf accelerate ipyfilechooser openai pandas matplotlib')
except NameError:
    import subprocess, sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q',
        'torch', 'transformers', 'sentence-transformers', 'faiss-cpu',
        'pymupdf', 'accelerate', 'ipyfilechooser', 'openai', 'pandas', 'matplotlib'])

In [ ]:
import os, time, json as _json
import torch
import faiss
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from pathlib import Path
from dataclasses import dataclass
from typing import List, Tuple
import fitz  # PyMuPDF

os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

def get_device():
    if torch.cuda.is_available():
        name = torch.cuda.get_device_name(0)
        mem  = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f'✔ CUDA GPU: {name} ({mem:.1f} GB)')
        return 'cuda', torch.float16
    elif torch.backends.mps.is_available():
        print('✔ Apple Silicon MPS')
        return 'mps', torch.float32
    else:
        print('⚠ CPU only')
        return 'cpu', torch.float32

try:
    import google.colab
    ENVIRONMENT = 'colab'
except ImportError:
    ENVIRONMENT = 'local'

DEVICE, DTYPE = get_device()
print(f'Environment: {ENVIRONMENT.upper()} | Device: {DEVICE}')

---
## Load Documents

In [ ]:
# =============================================================================
# CELL 1 — SELECT DOCUMENT SOURCE  (DO NOT CHANGE)
# =============================================================================
USE_GOOGLE_DRIVE = True
DOC_FOLDER = 'documents'
folder_chooser = None

if ENVIRONMENT == 'colab':
    if USE_GOOGLE_DRIVE:
        from google.colab import drive
        print('Mounting Google Drive...')
        drive.mount('/content/drive')
        print('✔ Google Drive mounted\n')
        try:
            from ipyfilechooser import FileChooser
            folder_chooser = FileChooser(
                path='/content/drive/MyDrive',
                title='Select your documents folder in Google Drive',
                show_only_dirs=True, select_default=True)
            print('📁 Select your documents folder below, then run Cell 2:')
            display(folder_chooser)
        except ImportError:
            DOC_FOLDER = '/content/drive/MyDrive/your_documents_folder'
            print(f"Edit DOC_FOLDER: '{DOC_FOLDER}'")
    else:
        from google.colab import files as colab_files
        os.makedirs(DOC_FOLDER, exist_ok=True)
        uploaded = colab_files.upload()
        for fn in uploaded:
            os.rename(fn, f'{DOC_FOLDER}/{fn}')
else:
    try:
        from ipyfilechooser import FileChooser
        folder_chooser = FileChooser(path=str(Path.home()),
            title='Select documents folder', show_only_dirs=True, select_default=True)
        display(folder_chooser)
    except ImportError:
        print(f'Using default folder: {DOC_FOLDER}')

In [ ]:
# =============================================================================
# CELL 2 — CONFIRM SELECTION  (DO NOT CHANGE)
# =============================================================================
if folder_chooser is not None and folder_chooser.selected_path:
    DOC_FOLDER = folder_chooser.selected_path
    print(f'✔ Using: {DOC_FOLDER}')
elif folder_chooser is not None:
    print('⚠ No folder selected — go back, select one, then re-run this cell.')
else:
    print(f'✔ Using: {DOC_FOLDER}')

In [5]:
import fitz  # PyMuPDF
from typing import List, Tuple

def load_text_file(filepath: str) -> str:
    """Load a plain text file."""
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
        return f.read()


def load_pdf_file(filepath: str) -> str:
    """
    Extract text from a PDF with embedded text.

    PyMuPDF reads the text layer directly.
    For scanned PDFs without embedded text, you'd need OCR.
    """
    doc = fitz.open(filepath)
    text_parts = []

    for page_num, page in enumerate(doc):
        text = page.get_text()
        if text.strip():
            # Add page marker for debugging/citation
            text_parts.append(f"\n[Page {page_num + 1}]\n{text}")

    doc.close()
    return "\n".join(text_parts)


def load_documents(doc_folder: str) -> List[Tuple[str, str]]:
    """Load all documents from a folder. Returns list of (filename, content)."""
    documents = []
    folder = Path(doc_folder)

    for filepath in folder.rglob("*"):
        try:
            if not filepath.is_file():
                continue
        except OSError:
            continue
        if filepath.suffix.lower() not in ('.pdf', '.txt', '.md', '.text'):
            continue
        try:
            if filepath.suffix.lower() == '.pdf':
                content = load_pdf_file(str(filepath))
            elif filepath.suffix.lower() in ['.txt', '.md', '.text']:
                content = load_text_file(str(filepath))
            else:
                continue

            if content.strip():
                documents.append((filepath.name, content))
                print(f"âœ“ Loaded: {filepath.name} ({len(content):,} chars)")
        except Exception as e:
            print(f"âœ— Error loading {filepath}: {e}")

    return documents

In [ ]:
# Load your documents
documents = load_documents(DOC_FOLDER)
print(f"\nLoaded {len(documents)} documents")

if len(documents) == 0:
    print("\nâš  No documents loaded! Please add PDF or TXT files to the documents folder.")

In [ ]:
# Inspect a document to verify loading worked
if documents:
    filename, content = documents[0]
    print(f"First document: {filename}")
    print(f"Total length: {len(content):,} characters")
    print(f"\nFirst 1000 characters:\n{'-'*40}")
    print(content[:1000])

---
## Build RAG Pipeline
Fixed chunk size (512) and overlap (128). Only the prompt template will vary.

In [ ]:
@dataclass
class Chunk:
    text: str
    source_file: str
    chunk_index: int
    start_char: int
    end_char: int


def chunk_text(text: str, source_file: str,
               chunk_size: int, chunk_overlap: int) -> list:
    chunks, start, idx = [], 0, 0
    while start < len(text):
        end = start + chunk_size
        if end < len(text):
            pb = text.rfind('\n\n', start + chunk_size // 2, end)
            if pb != -1:
                end = pb + 2
            else:
                sb = text.rfind('. ', start + chunk_size // 2, end)
                if sb != -1:
                    end = sb + 2
        s = text[start:end].strip()
        if s:
            chunks.append(Chunk(s, source_file, idx, start, end))
            idx += 1
        prev  = start
        start = end - chunk_overlap
        if start <= prev:
            start = end
    return chunks


def rebuild_pipeline(chunk_size: int = 512, chunk_overlap: int = 128):
    """Re-chunk documents, re-embed, rebuild FAISS index. Updates globals."""
    global all_chunks, index
    all_chunks = []
    for filename, content in documents:
        all_chunks.extend(
            chunk_text(content, filename,
                       chunk_size=chunk_size,
                       chunk_overlap=chunk_overlap)
        )
    chunk_embeddings = embed_model.encode(
        [c.text for c in all_chunks], show_progress_bar=True
    ).astype('float32')
    faiss.normalize_L2(chunk_embeddings)
    index = faiss.IndexFlatIP(EMBEDDING_DIM)
    index.add(chunk_embeddings)
    print(f'Rebuilt: {len(all_chunks)} chunks, '
          f'chunk_size={chunk_size}, chunk_overlap={chunk_overlap}')


all_chunks = []
index      = None
print('✔ Chunk + pipeline functions defined')

In [ ]:
from sentence_transformers import SentenceTransformer

EMBED_MODEL_NAME = 'all-MiniLM-L6-v2'
EMBEDDING_DIM    = 384

print(f'Loading embedding model: {EMBED_MODEL_NAME} ...')
embed_model = SentenceTransformer(EMBED_MODEL_NAME, device=DEVICE)
print('✔ Embedding model ready')

rebuild_pipeline(chunk_size=512, chunk_overlap=128)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

LLM_MODEL = 'Qwen/Qwen2.5-1.5B-Instruct'
print(f'Loading LLM: {LLM_MODEL} ...')
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)

if DEVICE == 'cuda':
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL, device_map='auto', dtype=DTYPE, trust_remote_code=True)
elif DEVICE == 'mps':
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL, dtype=DTYPE, trust_remote_code=True).to(DEVICE)
else:
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL, dtype=DTYPE, trust_remote_code=True)

print(f'✔ LLM loaded on {DEVICE}')

---
## Define Prompt Templates & Queries

In [ ]:
# =============================================================================
# THE 5 PROMPT TEMPLATE VARIANTS
# Each must contain {context} and {question} placeholders.
# =============================================================================

PROMPT_VARIANTS = [
    {
        'id'      : 'P1',
        'name'    : 'Minimal',
        'template': (
            'Context:\n{context}\n\n'
            'Question: {question}\n\n'
            'Answer:'
        )
    },
    {
        'id'      : 'P2',
        'name'    : 'Strict Grounding',
        'template': (
            'You are a helpful assistant.\n'
            'Answer ONLY based on the context below. '
            'If the answer is not in the context, say exactly: '
            '"The context does not contain enough information to answer this question."\n\n'
            'Context:\n{context}\n\n'
            'Question: {question}\n\n'
            'Answer:'
        )
    },
    {
        'id'      : 'P3',
        'name'    : 'Citation',
        'template': (
            'You are a helpful assistant.\n'
            'Answer the question using the context below. '
            'For every claim you make, quote the exact passage from the context '
            'that supports it, using quotation marks. '
            'If no passage supports a claim, do not make it.\n\n'
            'Context:\n{context}\n\n'
            'Question: {question}\n\n'
            'Answer (with inline quotes):'
        )
    },
    {
        'id'      : 'P4',
        'name'    : 'Permissive',
        'template': (
            'You are a helpful assistant with broad knowledge.\n'
            'Use the context below to help answer the question. '
            'You may also draw on your general knowledge to fill gaps '
            'or provide additional useful detail — but clearly distinguish '
            'what comes from the context vs. your own knowledge.\n\n'
            'Context:\n{context}\n\n'
            'Question: {question}\n\n'
            'Answer:'
        )
    },
    {
        'id'      : 'P5',
        'name'    : 'Structured Output',
        'template': (
            'You are a helpful assistant.\n'
            'Using the context below, answer in two clearly labelled sections:\n'
            '1. RELEVANT FACTS: List every fact from the context '
            'that is relevant to the question, as short bullet points.\n'
            '2. ANSWER: Synthesise those facts into a clear, '
            'direct answer to the question.\n\n'
            'Context:\n{context}\n\n'
            'Question: {question}\n\n'
            'Response:'
        )
    },
]


# =============================================================================
# 5 QUERIES (same set across all prompt variants)
# =============================================================================
QUERIES = [
    {
        'id'  : 'Q1',
        'type': 'narrow_factual',
        'text': 'What is the correct spark plug gap for a Model T Ford?'
    },
    {
        'id'  : 'Q2',
        'type': 'procedural',
        'text': 'How do I fix a slipping transmission band?'
    },
    {
        'id'  : 'Q3',
        'type': 'procedural',
        'text': 'How do I adjust the carburetor on a Model T?'
    },
    {
        'id'  : 'Q4',
        'type': 'broad_conceptual',
        'text': 'What oil should I use in a Model T engine?'
    },
    {
        'id'  : 'Q5',
        'type': 'unanswerable',
        'text': 'What is the fuel efficiency of the Model T in miles per gallon?'
    },
]

TOP_K = 5
print(f'✔ {len(PROMPT_VARIANTS)} prompt variants × {len(QUERIES)} queries = {len(PROMPT_VARIANTS)*len(QUERIES)} runs')
for p in PROMPT_VARIANTS:
    print(f"   [{p['id']}] {p['name']}")

---
## Run All Variant × Query Combinations

In [ ]:
def retrieve(question: str, top_k: int = 5) -> list:
    """Return list of (chunk, score) for top_k results."""
    q_emb = embed_model.encode([question]).astype('float32')
    faiss.normalize_L2(q_emb)
    scores, indices = index.search(q_emb, top_k)
    return [
        (all_chunks[i], float(s))
        for s, i in zip(scores[0], indices[0]) if i != -1
    ]


def generate_response(prompt: str, max_new_tokens: int = 350) -> str:
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(
        out[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    ).strip()


def run_variant(question: str, template: str, top_k: int = 5) -> dict:
    """Retrieve context, fill template, generate answer. Return stats dict."""
    retrieved = retrieve(question, top_k)
    context   = '\n\n---\n\n'.join(
        f'[Source: {c.source_file} | Score: {s:.3f}]\n{c.text}'
        for c, s in retrieved
    )
    prompt    = template.format(context=context, question=question)
    answer    = generate_response(prompt)
    top_score = retrieved[0][1] if retrieved else 0.0
    return {
        'answer'    : answer,
        'top_score' : round(top_score, 4),
        'context'   : context,   # kept for auto-eval
    }


print('✔ Helper functions defined')

In [ ]:
# ── Shared retrieved context per query (retrieval is the same for all variants) ──
# Pre-retrieve once per query so we don't re-run embedding 25 times.
shared_context = {}
for q in QUERIES:
    retrieved = retrieve(q['text'], TOP_K)
    shared_context[q['id']] = '\n\n---\n\n'.join(
        f'[Source: {c.source_file} | Score: {s:.3f}]\n{c.text}'
        for c, s in retrieved
    )
    top_score = retrieved[0][1] if retrieved else 0.0
    print(f"[{q['id']}] retrieved {len(retrieved)} chunks, top_score={top_score:.4f}")

print('\n✔ Context pre-retrieved for all queries')

In [ ]:
# ── Generate answers for all 25 combinations ──────────────────────────────────
raw_results = []   # one row per (variant × query)

for p in PROMPT_VARIANTS:
    print(f"\n{'='*60}")
    print(f"[{p['id']}] {p['name']}")
    print(f"{'='*60}")

    for q in QUERIES:
        print(f"  [{q['id']}] {q['text']}")

        context = shared_context[q['id']]
        prompt  = p['template'].format(context=context, question=q['text'])
        answer  = generate_response(prompt)

        print(f"  → {answer[:180]}...\n")

        raw_results.append({
            'prompt_id'     : p['id'],
            'prompt_name'   : p['name'],
            'query_id'      : q['id'],
            'query_type'    : q['type'],
            'question'      : q['text'],
            'answer'        : answer,
            'context'       : context,
            # eval columns filled in next cell
            'accuracy'      : None,
            'groundedness'  : None,
            'helpfulness'   : None,
            'citation_quality': None,
        })

    # Save after each variant — don't lose results if later ones crash
    pd.DataFrame(raw_results).to_csv('exercise10_raw_answers.csv', index=False)
    print(f'  ✔ Progress saved ({len(raw_results)} rows)')

print('\n✔ All 25 combinations complete')

---
## Auto-Evaluation via GPT-4o Mini

Each answer is scored 1–5 on four dimensions:

| Dimension | What it measures |
|---|---|
| **Accuracy** | Is the answer factually correct given the context? |
| **Groundedness** | Does every claim trace back to the context (no hallucination)? |
| **Helpfulness** | Would this answer actually help someone with their question? |
| **Citation quality** | Does the answer reference or quote the source material? |

In [ ]:
from openai import OpenAI

# ── API key ───────────────────────────────────────────────────────────────────
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print('✔ API key loaded from Colab secrets')
except Exception:
    OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', 'YOUR_KEY_HERE')
    if OPENAI_API_KEY == 'YOUR_KEY_HERE':
        print(' Set your OpenAI API key!')
    else:
        print(' API key loaded from environment variable')

oai_client = OpenAI(api_key=OPENAI_API_KEY)
JUDGE_MODEL = 'gpt-4o-mini'
print(f'Judge model: {JUDGE_MODEL}')

In [ ]:
def evaluate_answer(question: str, context: str, answer: str,
                    prompt_name: str) -> dict:
    """
    Ask GPT-4o Mini to score the answer on four dimensions (1–5).
    Returns a dict: {accuracy, groundedness, helpfulness, citation_quality}
    """
    judge_prompt = f"""You are an impartial evaluator of RAG (retrieval-augmented generation) answers.

A system used the following CONTEXT to answer a QUESTION.
The system prompt style was: "{prompt_name}".

QUESTION: {question}

CONTEXT (retrieved passages):
{context[:1500]}  ← truncated for brevity

ANSWER GENERATED:
{answer}

Score the answer on each dimension from 1 (very poor) to 5 (excellent):

accuracy        : Is the answer factually correct given the context?
groundedness    : Are all claims supported by the context (no hallucination)?
helpfulness     : Would this answer genuinely help someone asking the question?
citation_quality: Does the answer reference, quote, or attribute the source material?

Respond with ONLY valid JSON. Example:
{{"accuracy": 4, "groundedness": 5, "helpfulness": 3, "citation_quality": 2}}"""

    response = oai_client.chat.completions.create(
        model=JUDGE_MODEL,
        messages=[{'role': 'user', 'content': judge_prompt}],
        max_tokens=80,
        temperature=0,
    )
    raw = response.choices[0].message.content.strip()
    try:
        return _json.loads(raw)
    except Exception:
        return {'accuracy': None, 'groundedness': None,
                'helpfulness': None, 'citation_quality': None}


# ── Run evaluation on all 25 rows ─────────────────────────────────────────────
df = pd.DataFrame(raw_results)
print(f'Evaluating {len(df)} answers with {JUDGE_MODEL}...\n')

for i, row in df.iterrows():
    scores = evaluate_answer(
        question    = row['question'],
        context     = row['context'],
        answer      = row['answer'],
        prompt_name = row['prompt_name'],
    )
    df.at[i, 'accuracy']         = scores.get('accuracy')
    df.at[i, 'groundedness']     = scores.get('groundedness')
    df.at[i, 'helpfulness']      = scores.get('helpfulness')
    df.at[i, 'citation_quality'] = scores.get('citation_quality')

    print(f"  [{row['prompt_id']}×{row['query_id']}] "
          f"acc={scores.get('accuracy')}  "
          f"grnd={scores.get('groundedness')}  "
          f"help={scores.get('helpfulness')}  "
          f"cite={scores.get('citation_quality')}")
    time.sleep(0.4)

# Drop context column before saving (it's large and already in the raw CSV)
df_save = df.drop(columns=['context'])
df_save.to_csv('exercise10_evaluated_results.csv', index=False)
print('\n✔ Saved: exercise10_evaluated_results.csv')

---
## Results: Score Summary & Charts

In [ ]:
# ── Mean scores per prompt variant ────────────────────────────────────────────
EVAL_COLS = ['accuracy', 'groundedness', 'helpfulness', 'citation_quality']

df_num = df_save.copy()
for col in EVAL_COLS:
    df_num[col] = pd.to_numeric(df_num[col], errors='coerce')

pivot = (
    df_num.groupby(['prompt_id', 'prompt_name'])[EVAL_COLS]
          .mean()
          .round(2)
          .reset_index()
          .sort_values('prompt_id')
)
pivot['overall'] = pivot[EVAL_COLS].mean(axis=1).round(2)

print('=== MEAN SCORES BY PROMPT VARIANT (out of 5) ===')
pd.set_option('display.max_colwidth', 25)
display(pivot)

In [ ]:
# ── Chart 1: Grouped bar — all 4 dimensions per prompt variant ────────────────
variants = pivot['prompt_name'].tolist()
x        = np.arange(len(variants))
width    = 0.18
colours  = ['#4e79a7', '#f28e2b', '#59a14f', '#e15759']

fig, ax = plt.subplots(figsize=(12, 5))
for j, (col, colour) in enumerate(zip(EVAL_COLS, colours)):
    ax.bar(x + j * width, pivot[col], width,
           label=col.replace('_', ' ').title(), color=colour, alpha=0.85)

ax.set_xticks(x + width * 1.5)
ax.set_xticklabels([f"{r['prompt_id']}\n{r['prompt_name']}" for _, r in pivot.iterrows()],
                   fontsize=9)
ax.set_ylabel('Mean Score (1–5)', fontsize=10)
ax.set_ylim(0, 5.5)
ax.axhline(5, color='grey', linestyle=':', linewidth=0.7)
ax.legend(fontsize=9, loc='upper right')
ax.set_title('Mean Evaluation Scores by Prompt Variant', fontsize=12)
plt.tight_layout()
plt.savefig('exercise10_scores_grouped_bar.png', dpi=150, bbox_inches='tight')
plt.show()
print('✔ Saved: exercise10_scores_grouped_bar.png')

In [ ]:
# ── Chart 2: Heatmap — prompt variant × query ─────────────────────────────────
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for ax, col in zip(axes, EVAL_COLS):
    heat = df_num.pivot_table(
        index='prompt_name', columns='query_id',
        values=col, aggfunc='mean'
    )
    # Keep variants in order P1–P5
    order = [p['name'] for p in PROMPT_VARIANTS]
    heat  = heat.reindex(order)

    im = ax.imshow(heat.values, cmap='RdYlGn', vmin=1, vmax=5, aspect='auto')
    ax.set_xticks(range(len(heat.columns)))
    ax.set_xticklabels(heat.columns, fontsize=8)
    ax.set_yticks(range(len(heat.index)))
    ax.set_yticklabels(heat.index, fontsize=8)
    ax.set_title(col.replace('_', ' ').title(), fontsize=9)
    plt.colorbar(im, ax=ax, shrink=0.8)

    for i in range(len(heat.index)):
        for j in range(len(heat.columns)):
            val = heat.values[i, j]
            if not np.isnan(val):
                ax.text(j, i, f'{val:.1f}', ha='center', va='center',
                        fontsize=8,
                        color='white' if val < 2.5 else 'black')

plt.suptitle('Score Heatmap: Prompt Variant × Query  (1=poor, 5=excellent)',
             fontsize=11, y=1.02)
plt.tight_layout()
plt.savefig('exercise10_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()
print('✔ Saved: exercise10_heatmap.png')

In [ ]:
# ── Chart 3: Trade-off scatter — groundedness vs. helpfulness ─────────────────
means = (
    df_num.groupby('prompt_name')[['groundedness', 'helpfulness']]
          .mean().round(2)
          .reset_index()
)

fig, ax = plt.subplots(figsize=(7, 5))
colours_map = plt.get_cmap('tab10')

for i, row in means.iterrows():
    ax.scatter(row['groundedness'], row['helpfulness'],
               s=180, color=colours_map(i), zorder=3,
               edgecolors='black', linewidths=0.6)
    ax.annotate(
        row['prompt_name'],
        (row['groundedness'], row['helpfulness']),
        textcoords='offset points', xytext=(8, 4), fontsize=9
    )

ax.set_xlabel('Mean Groundedness', fontsize=11)
ax.set_ylabel('Mean Helpfulness', fontsize=11)
ax.set_xlim(0, 5.5)
ax.set_ylim(0, 5.5)
ax.axhline(3, color='grey', linestyle=':', linewidth=0.7)
ax.axvline(3, color='grey', linestyle=':', linewidth=0.7)
ax.set_title('Trade-off: Groundedness vs. Helpfulness\n'
             '(top-right = best of both worlds)', fontsize=11)
plt.tight_layout()
plt.savefig('exercise10_tradeoff.png', dpi=150, bbox_inches='tight')
plt.show()
print('✔ Saved: exercise10_tradeoff.png')

In [ ]:
# ── Print answers side-by-side for each query ─────────────────────────────────
pd.set_option('display.max_colwidth', 400)

for q in QUERIES:
    print(f"\n{'='*65}")
    print(f"[{q['id']}] ({q['type']}) {q['text']}")
    print(f"{'='*65}")
    sub = df_num[df_num['query_id'] == q['id']].sort_values('prompt_id')
    for _, row in sub.iterrows():
        print(f"\n  [{row['prompt_id']}] {row['prompt_name']}")
        print(f"  acc={row['accuracy']}  grnd={row['groundedness']}  "
              f"help={row['helpfulness']}  cite={row['citation_quality']}")
        print(f"  {row['answer'][:350]}")

---
## Documentation Questions

Fill in your observations after reviewing the scores and charts above.

### 1. Which prompt produces the most accurate answers?
*Look at the `accuracy` column in the summary table and the grouped bar chart.
Does strict grounding (P2) improve accuracy, or does it cause the model to refuse
answerable questions?*

---

### 2. Which prompt produces the most useful answers?
*Look at `helpfulness`. Is it the permissive prompt (P4) that scores highest?
Does the structured output (P5) improve usefulness by making answers easier to read?*

---

### 3. Is there a trade-off between strict grounding and helpfulness?
*Look at the groundedness vs. helpfulness scatter plot.
Does P2 (strict) land in the high-groundedness / low-helpfulness quadrant?
Does P4 (permissive) do the opposite?
Where does P5 (structured) land — can it achieve both?*

---

### 4. Bonus: How did each prompt handle the unanswerable question (Q5)?
*The Model T manual does not contain fuel efficiency data.
Did P2 (strict) correctly refuse? Did P4 (permissive) hallucinate an answer?
Did P1 (minimal) behave differently from P2 despite having no instructions?*

In [ ]:
# Download all outputs in Colab
output_files = [
    'exercise10_raw_answers.csv',
    'exercise10_evaluated_results.csv',
    'exercise10_scores_grouped_bar.png',
    'exercise10_heatmap.png',
    'exercise10_tradeoff.png',
]
try:
    from google.colab import files as colab_files
    for fname in output_files:
        if os.path.exists(fname):
            colab_files.download(fname)
            print(f' Downloading: {fname}')
        else:
            print(f' Not found (skipping): {fname}')
except ImportError:
    print('Files saved locally:')
    for fname in output_files:
        print(f'  {fname}')